<a href="https://colab.research.google.com/github/Harshkotkar/Deep-Learning/blob/main/rag_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community
!pip install langchain-google-genai
!pip install chromadb
!pip install -U langchain


In [2]:
!pip install pypdf

In [17]:
!pip install langchain-text-splitters
!pip install langchain.chains


ERROR: Could not find a version that satisfies the requirement langchain.chains (from versions: none)
ERROR: No matching distribution found for langchain.chains


In [4]:
import getpass
import os
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")


Enter your Google API key: ··········


In [5]:
#data ingestion
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("/content/kebo114.pdf")
docs=loader.load()

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_spliter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_spliter.split_documents(docs)


In [8]:
documents[:3]

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PageMaker 7.0', 'creationdate': '2022-10-14T15:17:04+00:00', 'author': 'Administrator', 'moddate': '2025-04-30T15:23:40+05:30', 'title': 'ch-14.pmd', 'source': '/content/kebo114.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='UNIT 5\nThe reductionist  approach to study of life forms resulted in increasing\nuse of physico-chemical concepts and techniques. Majority of these\nstudies employed either surviving tissue model or straightaway cell-\nfree systems. An explosion of knowledge resulted in molecular biology.\nMolecular physiology became almost synonymous with biochemistry\nand biophysics. However, it is now being incr easingly r\nealised that\nneither a purely organismic approach nor a purely reductionistic\nmolecular approach would reveal the truth about biological processes\nor living phenomena. Systems biology makes us believe that all living\nphenomena are emergent properties due to interac

In [9]:
# #vector embeding
# from langchain_community.embeddings import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma

# db=Chroma.from_documents(documents[:10],OpenAIEmbeddings())

In [10]:
#vector embeding
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Create the embedding model
model_name = "BAAI/bge-base-en-v1.5" #768 dimension for base, 384 dimension for small,1024 for large
model_kwargs = {'device': 'cuda'}  # or 'cuda' for GPU and cpu for if no gpu avalable
encode_kwargs = {'normalize_embeddings': True}  # better for similarity search

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Create your vectorstore
db = Chroma.from_documents(documents[:20], embeddings)

/tmp/ipython-input-1728801213.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
query='who is Alfonso Corti?'
result=db.similarity_search(query)
result

[Document(metadata={'source': '/content/kebo114.pdf', 'title': 'ch-14.pmd', 'page_label': '2', 'moddate': '2025-04-30T15:23:40+05:30', 'author': 'Administrator', 'creator': 'PageMaker 7.0', 'total_pages': 12, 'page': 1, 'producer': 'GPL Ghostscript 8.15', 'creationdate': '2022-10-14T15:17:04+00:00'}, page_content='ALFONSO CORTI, Italian anatomist, was born in 1822. Corti began\nhis scientific career studying the cardiovascular systems of\nreptiles. Later, he tur ned his attention to the mammalian\nauditory system. In 1851, he published a paper describing a\nstructure located on the basilar membrane of the cochlea\ncontaining hair cells that convert sound vibrations into nerve\nimpulses, the organ of Corti. He died in the year 1888.\nAlfonso Corti\n(1822 – 1888)\nReprint 2025-26'),
 Document(metadata={'title': 'ch-14.pmd', 'moddate': '2025-04-30T15:23:40+05:30', 'creationdate': '2022-10-14T15:17:04+00:00', 'page_label': '1', 'total_pages': 12, 'author': 'Administrator', 'page': 0, 'prod

In [12]:
query='who is Alfonso Corti?'
result = db.similarity_search_with_relevance_scores(query)
highest_similarity = max(result, key=lambda x: x[1])
document, score = highest_similarity

print(f"Highest similarity score: {score:.4f}")
print(f"Content: {document.page_content}")

Highest similarity score: 0.5158
Content: ALFONSO CORTI, Italian anatomist, was born in 1822. Corti began
his scientific career studying the cardiovascular systems of
reptiles. Later, he tur ned his attention to the mammalian
auditory system. In 1851, he published a paper describing a
structure located on the basilar membrane of the cochlea
containing hair cells that convert sound vibrations into nerve
impulses, the organ of Corti. He died in the year 1888.
Alfonso Corti
(1822 – 1888)
Reprint 2025-26


In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0)
llm

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-exp', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7e9eac33a7e0>, default_metadata=(), model_kwargs={})

In [14]:
from langchain_core.prompts import ChatPromptTemplate
context = document.page_content
prompt=ChatPromptTemplate.from_template(f'''
Answer the following question based only on the provided context.
think step by step before providing a detail answer.
<context>
{context}
<context>
Question:{input}
''')

In [20]:
!pip install langchain-core


In [25]:
from langchain_core.runnables import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)


ImportError: cannot import name 'create_stuff_documents_chain' from 'langchain_core.runnables' (/usr/local/lib/python3.12/dist-packages/langchain_core/runnables/__init__.py)

In [ ]:

"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well.
 https://python.langchain.com/docs/modules/data_connection/retrievers/
"""

retriever=db.as_retriever()
retriever

In [ ]:
document_chain = create_stuff_documents_chain(llm, prompt)
response = document_chain.invoke([document])
print(response)


In [24]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

document_chain = (
    RunnablePassthrough.assign(context=lambda x: x["documents"])
    | prompt
    | llm
    | StrOutputParser()
)

response = document_chain.invoke({
    "documents": document.page_content,
    "input": "Who is Alfonso Corti?"
})
print(response)


Here's a step-by-step breakdown to answer the question based solely on the provided text:

1.  **Identify the subject:** The text is about Alfonso Corti.

2.  **Focus on the question's topic:** The question asks about Alfonso Corti's life and work.

3.  **Extract relevant information:**
    *   Born in 1822.
    *   Italian anatomist.
    *   Studied cardiovascular systems of reptiles.
    *   Later studied the mammalian auditory system.
    *   In 1851, published a paper describing the organ of Corti (structure on the basilar membrane of the cochlea containing hair cells that convert sound vibrations into nerve impulses).
    *   Died in 1888.

**Answer:**

Alfonso Corti was an Italian anatomist born in 1822. He initially studied the cardiovascular systems of reptiles but later focused on the mammalian auditory system. In 1851, he published a paper describing the organ of Corti, a structure in the cochlea responsible for converting sound vibrations into nerve impulses. He died in 1888